In this guided project, we will walk through how to normalize our single table into multiple tables and how to create relations between them.

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("nominations.db")

In [3]:
pd.read_sql_query("SELECT * FROM nominations LIMIT 20;", conn)

,Year,Category,Nominee,Won,Movie,Character
0,2010,Actor -- Leading Role,Javier Bardem,0,Biutiful,Uxbal
1,2010,Actor -- Leading Role,Jeff Bridges,0,True Grit,Rooster Cogburn
2,2010,Actor -- Leading Role,Jesse Eisenberg,0,The Social Network,Mark Zuckerberg
3,2010,Actor -- Leading Role,Colin Firth,1,The King's Speech,King George VI
4,2010,Actor -- Leading Role,James Franco,0,127 Hours,Aron Ralston
5,2010,Actor -- Supporting Role,Christian Bale,1,The Fighter,Dicky Eklund
6,2010,Actor -- Supporting Role,John Hawkes,0,Winter's Bone,Teardrop
7,2010,Actor -- Supporting Role,Jeremy Renner,0,The Town,James Coughlin
8,2010,Actor -- Supporting Role,Mark Ruffalo,0,The Kids Are All Right,Paul
9,2010,Actor -- Supporting Role,Geoffrey Rush,0,The King's Speech,Lionel Logue


In [4]:
pd.read_sql_query('PRAGMA table_info(nominations);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Year,INTEGER,0,None,0
1,1,Category,TEXT,0,None,0
2,2,Nominee,TEXT,0,None,0
3,3,Won,INTEGER,0,None,0
4,4,Movie,TEXT,0,None,0
5,5,Character,TEXT,0,None,0


Let's create a ceremonies table that contains the **Year** and **Host** for each ceremony and then set up a **one-to-many** relationship between ceremonies and nominations.

The Python sqlite3 library comes with an [executemany](https://docs.python.org/3/library/sqlite3.html#sqlite3.Cursor.executemany) method that let's us easily mass insert records into a table.

In [5]:
# Create the ceremonies table.
conn.execute('CREATE TABLE ceremonies(\
id integer PRIMARY KEY,\
Year integer,\
Host text\
);')

query = 'PRAGMA table_info(ceremonies);'
pd.read_sql_query(query, conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,Year,integer,0,None,0
2,2,Host,text,0,None,0


In [6]:
# Create the list of tuples, years_hosts,
# that contains the values for the rows we want to insert into the ceremonies table.
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

# Use the Connection method executemany to insert the values into the ceremonies table.
conn.executemany('INSERT INTO ceremonies (Year, Host) VALUES (?,?);', years_hosts)

pd.read_sql_query("SELECT * FROM ceremonies LIMIT 10;", conn)

,id,Year,Host
0,1,2010,Steve Martin
1,2,2009,Hugh Jackman
2,3,2008,Jon Stewart
3,4,2007,Ellen DeGeneres
4,5,2006,Jon Stewart
5,6,2005,Chris Rock
6,7,2004,Billy Crystal
7,8,2003,Steve Martin
8,9,2002,Whoopi Goldberg
9,10,2001,Steve Martin


In [7]:
conn.execute('DELETE FROM ceremonies;')

pd.read_sql_query("SELECT * FROM ceremonies LIMIT 10;", conn)

,id,Year,Host


In [8]:
conn.execute('INSERT INTO ceremonies (Year, Host) \
VALUES \
(2010, "Steve Martin"),\
(2009, "Hugh Jackman"),\
(2008, "Jon Stewart"),\
(2007, "Ellen DeGeneres"),\
(2006, "Jon Stewart"),\
(2005, "Chris Rock"),\
(2004, "Billy Crystal"),\
(2003, "Steve Martin"),\
(2002, "Whoopi Goldberg"),\
(2001, "Steve Martin"),\
(2000, "Billy Crystal")\
;')

pd.read_sql_query("SELECT * FROM ceremonies LIMIT 10;", conn)

,id,Year,Host
0,1,2010,Steve Martin
1,2,2009,Hugh Jackman
2,3,2008,Jon Stewart
3,4,2007,Ellen DeGeneres
4,5,2006,Jon Stewart
5,6,2005,Chris Rock
6,7,2004,Billy Crystal
7,8,2003,Steve Martin
8,9,2002,Whoopi Goldberg
9,10,2001,Steve Martin


To prevent us from inserting rows with nonexisting foreign key values, we need to turn on foreign key constraints by running the following query:

**PRAGMA foreign_keys = ON;**

#### The above query needs to be run every time you connect to a database where you'll be inserting foreign keys.

In [9]:
# Turn on foreign key constraints.
conn.execute('PRAGMA foreign_keys=ON;')

The next step is to remove the Year column from nominations and add a new column, ceremony_id, that contains the foreign key reference to the id column in the ceremonies table.

Unfortunately, we can't remove columns from an existing table in SQLite or change its schema.

The only alterations we can make to an existing table are renaming it or adding a new column.

This means that we can't just remove the Year column from nominations and add the ceremony_id column.

We need to instead:
* create a new table nominations_two with the schema we want,
* populate nominations_two with the records we want,
* delete the original nominations table,
* rename nominations_two to nominations.

In [10]:
# The query to create the nominations_two table.
conn.execute('CREATE TABLE nominations_two(\
id integer PRIMARY KEY,\
category text,\
nominee text,\
movie text,\
character text,\
won integer,\
ceremony_id integer,\
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id)\
);')

pd.read_sql_query('PRAGMA table_info(nominations_two);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,category,text,0,None,0
2,2,nominee,text,0,None,0
3,3,movie,text,0,None,0
4,4,character,text,0,None,0
5,5,won,integer,0,None,0
6,6,ceremony_id,integer,0,None,0


In [11]:
# The query that returns the records from the nominations table and assign the results set to joined_nominations.
query = 'SELECT ceremonies.id as ceremony_id,\
nominations.category,nominations.nominee,nominations.movie,nominations.character,nominations.won \
FROM nominations \
INNER JOIN ceremonies ON nominations.year==ceremonies.year;'

joined_nominations = conn.execute(query).fetchall()

pd.read_sql_query(query, conn)[:10]

,ceremony_id,Category,Nominee,Movie,Character,Won
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0
1,1,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0
2,1,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0
3,1,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1
4,1,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0
5,1,Actor -- Supporting Role,Christian Bale,The Fighter,Dicky Eklund,1
6,1,Actor -- Supporting Role,John Hawkes,Winter's Bone,Teardrop,0
7,1,Actor -- Supporting Role,Jeremy Renner,The Town,James Coughlin,0
8,1,Actor -- Supporting Role,Mark Ruffalo,The Kids Are All Right,Paul,0
9,1,Actor -- Supporting Role,Geoffrey Rush,The King's Speech,Lionel Logue,0


In [12]:
# A placeholder insert query that can insert values into nominations_two
# and assign this query to insert_nominations_two.
insert_nominations_two = 'INSERT INTO nominations_two \
(ceremony_id,category,nominee,movie,character,won) \
VALUES (?,?,?,?,?,?);'

conn.executemany(insert_nominations_two, joined_nominations)

pd.read_sql_query("SELECT * FROM nominations_two LIMIT 10;", conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1
5,6,Actor -- Supporting Role,Christian Bale,The Fighter,Dicky Eklund,1,1
6,7,Actor -- Supporting Role,John Hawkes,Winter's Bone,Teardrop,0,1
7,8,Actor -- Supporting Role,Jeremy Renner,The Town,James Coughlin,0,1
8,9,Actor -- Supporting Role,Mark Ruffalo,The Kids Are All Right,Paul,0,1
9,10,Actor -- Supporting Role,Geoffrey Rush,The King's Speech,Lionel Logue,0,1


In [13]:
# The query that deletes the nominations table from the database.
conn.execute('DROP TABLE nominations;')

In [14]:
# The query that renames nominations_two to nominations.
conn.execute('ALTER TABLE nominations_two RENAME TO nominations;')

In [15]:
pd.read_sql_query("SELECT * FROM nominations LIMIT 10;", conn)

,id,category,nominee,movie,character,won,ceremony_id
0,1,Actor -- Leading Role,Javier Bardem,Biutiful,Uxbal,0,1
1,2,Actor -- Leading Role,Jeff Bridges,True Grit,Rooster Cogburn,0,1
2,3,Actor -- Leading Role,Jesse Eisenberg,The Social Network,Mark Zuckerberg,0,1
3,4,Actor -- Leading Role,Colin Firth,The King's Speech,King George VI,1,1
4,5,Actor -- Leading Role,James Franco,127 Hours,Aron Ralston,0,1
5,6,Actor -- Supporting Role,Christian Bale,The Fighter,Dicky Eklund,1,1
6,7,Actor -- Supporting Role,John Hawkes,Winter's Bone,Teardrop,0,1
7,8,Actor -- Supporting Role,Jeremy Renner,The Town,James Coughlin,0,1
8,9,Actor -- Supporting Role,Mark Ruffalo,The Kids Are All Right,Paul,0,1
9,10,Actor -- Supporting Role,Geoffrey Rush,The King's Speech,Lionel Logue,0,1


In [16]:
pd.read_sql_query("SELECT * FROM ceremonies LIMIT 10;", conn)

,id,Year,Host
0,1,2010,Steve Martin
1,2,2009,Hugh Jackman
2,3,2008,Jon Stewart
3,4,2007,Ellen DeGeneres
4,5,2006,Jon Stewart
5,6,2005,Chris Rock
6,7,2004,Billy Crystal
7,8,2003,Steve Martin
8,9,2002,Whoopi Goldberg
9,10,2001,Steve Martin


#### Creating a join table.

In [17]:
# Create the 3 tables we need to model the relationship between movies and actors.
# The query to create the nominations_two table.
conn.execute('CREATE TABLE movies(\
id integer PRIMARY KEY,\
movie text\
);')

pd.read_sql_query('PRAGMA table_info(movies);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,movie,text,0,None,0


In [18]:
conn.execute('CREATE TABLE actors(\
id integer PRIMARY KEY,\
actor text\
);')

pd.read_sql_query('PRAGMA table_info(actors);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,actor,text,0,None,0


In [19]:
conn.execute('CREATE TABLE movies_actors(\
id integer PRIMARY KEY,\
movie_id integer REFERENCES movies(id),\
actor_id integer REFERENCES actors(id)\
);')

pd.read_sql_query('PRAGMA table_info(movies_actors);', conn)

,cid,name,type,notnull,dflt_value,pk
0,0,id,integer,0,None,1
1,1,movie_id,integer,0,None,0
2,2,actor_id,integer,0,None,0


#### Populating the movies and actors tables.

In [20]:
movies = conn.execute("SELECT DISTINCT movie FROM nominations;").fetchall()

conn.executemany('INSERT INTO movies (movie) VALUES (?);', movies)

pd.read_sql_query("SELECT * FROM movies LIMIT 10;", conn)

,id,movie
0,1,Biutiful
1,2,True Grit
2,3,The Social Network
3,4,The King's Speech
4,5,127 Hours
5,6,The Fighter
6,7,Winter's Bone
7,8,The Town
8,9,The Kids Are All Right
9,10,Rabbit Hole


In [21]:
actors = conn.execute("SELECT DISTINCT nominee FROM nominations;").fetchall()

conn.executemany('INSERT INTO actors (actor) VALUES (?);', actors)

pd.read_sql_query("SELECT * FROM actors LIMIT 10;", conn)

,id,actor
0,1,Javier Bardem
1,2,Jeff Bridges
2,3,Jesse Eisenberg
3,4,Colin Firth
4,5,James Franco
5,6,Christian Bale
6,7,John Hawkes
7,8,Jeremy Renner
8,9,Mark Ruffalo
9,10,Geoffrey Rush


In [22]:
conn.execute('DELETE FROM movies;')
conn.execute('DELETE FROM actors;')

In [23]:
conn.execute('INSERT INTO movies (movie) SELECT DISTINCT movie FROM nominations;')

pd.read_sql_query("SELECT * FROM movies LIMIT 10;", conn)

,id,movie
0,1,Biutiful
1,2,True Grit
2,3,The Social Network
3,4,The King's Speech
4,5,127 Hours
5,6,The Fighter
6,7,Winter's Bone
7,8,The Town
8,9,The Kids Are All Right
9,10,Rabbit Hole


In [24]:
conn.execute('INSERT INTO actors (actor) SELECT DISTINCT nominee FROM nominations;')

pd.read_sql_query("SELECT * FROM actors LIMIT 10;", conn)

,id,actor
0,1,Javier Bardem
1,2,Jeff Bridges
2,3,Jesse Eisenberg
3,4,Colin Firth
4,5,James Franco
5,6,Christian Bale
6,7,John Hawkes
7,8,Jeremy Renner
8,9,Mark Ruffalo
9,10,Geoffrey Rush


#### Populating a join table.

In [25]:
movie_actor_pairs = conn.execute('SELECT DISTINCT movie,nominee FROM nominations;').fetchall()

conn.executemany('INSERT INTO movies_actors (movie_id,actor_id) \
VALUES ((SELECT id FROM movies WHERE movie==?),(SELECT id FROM actors WHERE actor==?))\
;', movie_actor_pairs)

pd.read_sql_query("SELECT * FROM movies_actors LIMIT 20;", conn)

,id,movie_id,actor_id
0,1,1,1
1,2,2,2
2,3,3,3
3,4,4,4
4,5,5,5
5,6,6,6
6,7,7,7
7,8,8,8
8,9,9,9
9,10,4,10


In [26]:
pd.read_sql_query("SELECT movies.movie,actors.actor FROM movies_actors \
INNER JOIN movies ON movies_actors.movie_id==movies.id \
INNER JOIN actors ON movies_actors.actor_id==actors.id \
WHERE movies.movie=='The Fighter';", conn)

,movie,actor
0,The Fighter,Christian Bale
1,The Fighter,Amy Adams
2,The Fighter,Melissa Leo
